# 第7章 数据规整化：清理、转换、合并、重塑

## 合并数据集

### 数据库风格的DataFrame合并

- 数据集的合并（merge）或连接（join）运算是通过一个或多个键将行链接起来的

In [1]:
import pandas as pd
from pandas import Series, DataFrame
import numpy as np

In [2]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [3]:
df2 = DataFrame({'key': ['a', 'b', 'd'], 'data2': range(3)})
df2

,data2,key
0,0,a
1,1,b
2,2,d


In [4]:
# 没有指明要用哪个列进行连接，merge会将重叠列的列名当做键
pd.merge(df1, df2)

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [5]:
# 显式指定用哪个列进行连接
pd.merge(df1, df2, on='key')

,data1,key,data2
0,0,b,1
1,1,b,1
2,6,b,1
3,2,a,0
4,4,a,0
5,5,a,0


In [6]:
df3 = DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df3

,data1,lkey
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,a
6,6,b


In [7]:
df4 = DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})
df4

,data2,rkey
0,0,a
1,1,b
2,2,d


In [8]:
# 如果两个列名的对象不一样，也可以分别指定
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,data1,lkey,data2,rkey
0,0,b,1,b
1,1,b,1,b
2,6,b,1,b
3,2,a,0,a
4,4,a,0,a
5,5,a,0,a


In [9]:
# merge默认做的是“inner”连接，结果中的键是交集，其他方式还有“left”、“right”以及“outer”
pd.merge(df1, df2, how='outer')

,data1,key,data2
0,0.0,b,1.0
1,1.0,b,1.0
2,6.0,b,1.0
3,2.0,a,0.0
4,4.0,a,0.0
5,5.0,a,0.0
6,3.0,c,NaN
7,NaN,d,2.0


In [10]:
df1 = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df1

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [11]:
df2 = DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})
df2

,data2,key
0,0,a
1,1,b
2,2,a
3,3,b
4,4,d


In [12]:
# 多对多的连接，产生的是行的笛卡尔积（左边3个b，右边2个b，最终结果有6个b）
pd.merge(df1, df2, on='key', how='left')

,data1,key,data2
0,0,b,1.0
1,0,b,3.0
2,1,b,1.0
3,1,b,3.0
4,2,a,0.0
5,2,a,2.0
6,3,c,NaN
7,4,a,0.0
8,4,a,2.0
9,5,b,1.0


In [13]:
left = DataFrame({'key1': ['foo', 'foo', 'bar'], 'key2': ['one', 'two', 'one'], 'lval': [1, 2, 3]})
left

,key1,key2,lval
0,foo,one,1
1,foo,two,2
2,bar,one,3


In [14]:
right = DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'], 'key2': ['one', 'one', 'one', 'two'], 'rval': [4, 5, 6, 7]})
right

,key1,key2,rval
0,foo,one,4
1,foo,one,5
2,bar,one,6
3,bar,two,7


In [15]:
# 要根据多个键进行合并，传入一个由列名组成的列表即可（多个键形成一系列元组，并将其当做单个连接键）
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


In [16]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [17]:
# 对于重复列名的处理，merge提供了一个suffixes参数
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### 索引上的合并

- 有时候，DataFrame的连接键位于其索引中，在这种情况下，你可以传入left_index=True或right_index=True（或两个都传）以说明索引应该被用作连接键

In [18]:
left1 = DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [19]:
right1 = DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
right1

,group_val
a,3.5
b,7.0


In [20]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


In [21]:
# 默认的merge方法是求取交集，可以通过外连接指定求并集
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


- DataFrame还有一个join实例方法，它能更为方便地实现按索引合并，它还可用于合并多个带有相同或相似索引的DataFrame对象，而不管它们之间有没有重叠的列

In [22]:
left1.join(right1, how='outer', on='key')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


### 轴向连接

- 另一种数据合并运算也被称作连接（concatenation）、绑定（binding）或堆叠（stacking），NumPy有一个用于合并原始NumPy数组的concatenation函数

In [23]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [24]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

- pandas的concat函数可以提供类似功能

In [25]:
s1 = Series([0, 1], index=['a', 'b'])
s1

a    0
b    1
dtype: int64

In [26]:
s2 = Series([2, 3, 4], index=['c', 'd', 'e'])
s2

c    2
d    3
e    4
dtype: int64

In [27]:
s3 = Series([5, 6], index=['f', 'g'])
s3

f    5
g    6
dtype: int64

In [28]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

- 默认情况下，concat是在axis=0上工作的，最终产生一个新的Series，如果传入axis=1，则结果就会变成一个DataFrame

In [29]:
# 这种情况下，另外一条轴上没有重叠，从索引的有序并集（外连接）上就可以看出来
pd.concat([s1, s2, s3], axis=1, sort=False)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


In [30]:
s4 = pd.concat([s1 * 5, s3])
s4

a    0
b    5
f    5
g    6
dtype: int64

In [31]:
pd.concat([s1, s4], axis=1, sort=True)

,0,1
a,0.0,0
b,1.0,5
f,NaN,5
g,NaN,6


In [32]:
# 取交集
pd.concat([s1, s4], axis=1, sort=True, join='inner')

,0,1
a,0,0
b,1,5


In [33]:
# 通过join_axes指定要在其它轴上使用的索引
pd.concat([s1, s4], axis=1, join_axes=[['a', 'c', 'b', 'e']])

,0,1
a,0.0,0.0
c,NaN,NaN
b,1.0,5.0
e,NaN,NaN


In [34]:
df1 = DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df1

,a,b,c,d
0,0.222607,-0.167062,-0.673669,0.011647
1,0.056135,0.186694,-1.641464,-0.300787
2,1.391311,-0.092176,-0.682352,-0.002622


In [35]:
df2 = DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df2

,b,d,a
0,0.078301,-0.869863,-1.330816
1,0.027536,0.196049,0.021546


In [36]:
pd.concat([df1, df2], sort=True)

,a,b,c,d
0,0.222607,-0.167062,-0.673669,0.011647
1,0.056135,0.186694,-1.641464,-0.300787
2,1.391311,-0.092176,-0.682352,-0.002622
0,-1.330816,0.078301,NaN,-0.869863
1,0.021546,0.027536,NaN,0.196049


In [37]:
pd.concat([df1, df2], sort=True, ignore_index=True)

,a,b,c,d
0,0.222607,-0.167062,-0.673669,0.011647
1,0.056135,0.186694,-1.641464,-0.300787
2,1.391311,-0.092176,-0.682352,-0.002622
3,-1.330816,0.078301,NaN,-0.869863
4,0.021546,0.027536,NaN,0.196049


### 合并重叠数据

- 还有一种数据组合问题不能用简单的合并（merge）或连接（concatenation）运算来处理

In [38]:
a = Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index=['f', 'e', 'd', 'c', 'b', 'a'])
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [39]:
b = Series(np.arange(len(a), dtype=np.float64), index=['f', 'e', 'd', 'c', 'b', 'a'])
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [40]:
b[-1] = np.nan
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [41]:
# NumPy的where函数，用于表达一种矢量化的if-else
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

- Series有一个combine_first方法，实现where一样的功能，而且会进行数据对齐

In [42]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

- 对于DataFrame，combine_first自然也会在列上做同样的事情，因此你可以将其看做：用参数对象中的数据为调用者对象的缺失数据“打补丁”

In [43]:
df1 = DataFrame({'a': [1, np.nan, 5, np.nan], 'b': [np.nan, 2, np.nan, 6], 'c': range(2, 18, 4)})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [44]:
df2 = DataFrame({'a': [5, 4, np.nan, 3, 7], 'b': [np.nan, 3, 4, 6, 8]})
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [45]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 重塑和轴向旋转

- 有许多用于重新排列表格型数据的基础运算，这些函数也称作重塑（reshape）或轴向旋转（pivot）运算

### 重塑层次化索引

- 层次化索引为DataFrame数据的重排任务提供了一种具有良好一致性的方式，主要功能有二：stack，将数据的列旋转为行；unstack，将数据的行旋转为列

In [46]:
data = DataFrame(np.arange(6).reshape((2, 3)), index=pd.Index(['Ohio', 'Colorado'], name='state'), columns=pd.Index(['one', 'two', 'three'], name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


In [47]:
# 使用该数据的stack方法即可将列转换成行，得到一个Series
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [48]:
# 对于一个层次化索引的Series，你可以用unstack将其重排为一个DataFrame
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


## 数据转换

### 移除重复数据

In [49]:
data = DataFrame({'k1': ['one'] * 3 + ['two'] * 4, 'k2': [1, 1, 2, 3, 3, 4, 4]})
data

,k1,k2
0,one,1
1,one,1
2,one,2
3,two,3
4,two,3
5,two,4
6,two,4


- DataFrame的duplicated方法返回一个布尔型Series，表示各行是否是重复行

In [50]:
data.duplicated()

0    False
1     True
2    False
3    False
4     True
5    False
6     True
dtype: bool

- 还有一个与此相关的drop_duplicates方法，它用于返回一个移除了重复行的DataFrame

In [51]:
data.drop_duplicates()

,k1,k2
0,one,1
2,one,2
3,two,3
5,two,4


- 这两个方法默认会判断全部列，你也可以指定部分列进行重复项判断

In [52]:
data['v1'] = range(7)
data

,k1,k2,v1
0,one,1,0
1,one,1,1
2,one,2,2
3,two,3,3
4,two,3,4
5,two,4,5
6,two,4,6


In [53]:
data.drop_duplicates(['k1'])

,k1,k2,v1
0,one,1,0
3,two,3,3


- duplicated和drop_duplicates默认保留第一个出现的值组合，传入keep='last'则保留最后一个

In [54]:
data.drop_duplicates(['k1', 'k2'], keep='last')

,k1,k2,v1
1,one,1,1
2,one,2,2
4,two,3,4
6,two,4,6


### 利用函数或映射进行数据转换

- 在对数据集进行转换时，你可能希望根据数组、Series或DataFrame列中的值来实现转换工作，map函数可以满足需求

In [55]:
data = DataFrame({'food': ['bacon', 'pulled pork', 'bacon', 'Pastrami', 'corned beef', 'Bacon', 'pastrami', 'honey ham', 'nova lox'], 'ounces': [4, 3, 12, 6, 7.5, 8, 3, 5, 6]})
data

,food,ounces
0,bacon,4.0
1,pulled pork,3.0
2,bacon,12.0
3,Pastrami,6.0
4,corned beef,7.5
5,Bacon,8.0
6,pastrami,3.0
7,honey ham,5.0
8,nova lox,6.0


In [56]:
meat_to_animal = {'bacon': 'pig', 'pulled pork': 'pig', 'pastrami': 'cow', 'corned beef': 'cow', 'honey ham': 'pig', 'nova lox': 'salmon'}
data['animal'] = data['food'].map(lambda x: meat_to_animal[x.lower()])
data

,food,ounces,animal
0,bacon,4.0,pig
1,pulled pork,3.0,pig
2,bacon,12.0,pig
3,Pastrami,6.0,cow
4,corned beef,7.5,cow
5,Bacon,8.0,pig
6,pastrami,3.0,cow
7,honey ham,5.0,pig
8,nova lox,6.0,salmon


### 替换值

- 利用fillna方法填充缺失数据可以看做值替换的一种特殊情况，前面提到的map可用于修改对象的数据子集，而replace则提供了一种实现该功能的更简单、更灵活的方式

In [57]:
data = Series([1, -999, 2, -999, -1000, 3])
data

0       1
1    -999
2       2
3    -999
4   -1000
5       3
dtype: int64

In [58]:
data.replace(-999, np.nan)

0       1.0
1       NaN
2       2.0
3       NaN
4   -1000.0
5       3.0
dtype: float64

In [59]:
data.replace([-999, -1000], np.nan)

0    1.0
1    NaN
2    2.0
3    NaN
4    NaN
5    3.0
dtype: float64

In [60]:
data.replace([-999, -1000], [np.nan, 0])

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

In [61]:
data.replace({-999: np.nan, -1000: 0})

0    1.0
1    NaN
2    2.0
3    NaN
4    0.0
5    3.0
dtype: float64

### 重命名轴索引

- 跟Series中的值一样，轴标签也可以通过函数或映射进行转换，从而得到一个新对象，轴还可以被就地修改，而无需新建一个数据结构

In [62]:
data = DataFrame(np.arange(12).reshape((3, 4)), index=['Ohio', 'Colorado', 'New York'], columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [63]:
data.index.map(str.upper)

Index([u'OHIO', u'COLORADO', u'NEW YORK'], dtype='object')

In [64]:
# 就地修改
data.index = data.index.map(str.upper)
data

,one,two,three,four
OHIO,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [65]:
# 不进行就地修改，生成新对象
data.rename(index=str.title, columns=str.upper)

,ONE,TWO,THREE,FOUR
Ohio,0,1,2,3
Colorado,4,5,6,7
New York,8,9,10,11


In [66]:
# rename还支持字典参数
data.rename(index={'OHIO': 'INDIANA'}, columns={'three': 'peekaboo'})

,one,two,peekaboo,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


In [67]:
# rename方法也支持就地修改
data.rename(index={'OHIO': 'INDIANA'}, inplace=True)
data

,one,two,three,four
INDIANA,0,1,2,3
COLORADO,4,5,6,7
NEW YORK,8,9,10,11


### 离散化和面元划分

- 为了便于分析，连续数据常常被离散化或拆分为“面元”（bin）

In [68]:
# pandas返回的是一个特殊的Categorical对象，你可以将其看做一组表示面元名称的字符串
ages = [20, 22, 25, 27, 21, 23, 37, 31, 61, 45, 41, 32]
bins = [18, 25, 35, 60, 100]
cats = pd.cut(ages, bins)
cats

[(18, 25], (18, 25], (18, 25], (25, 35], (18, 25], ..., (25, 35], (60, 100], (35, 60], (35, 60], (25, 35]]
Length: 12
Categories (4, interval[int64]): [(18, 25] < (25, 35] < (35, 60] < (60, 100]]

In [69]:
cats.categories

IntervalIndex([(18, 25], (25, 35], (35, 60], (60, 100]]
              closed='right',
              dtype='interval[int64]')

In [70]:
# 可以设置闭区间的方向
cats = pd.cut(ages, bins, right=False)
cats

[[18, 25), [18, 25), [25, 35), [25, 35), [18, 25), ..., [25, 35), [60, 100), [35, 60), [35, 60), [25, 35)]
Length: 12
Categories (4, interval[int64]): [[18, 25) < [25, 35) < [35, 60) < [60, 100)]

In [71]:
# 可以设置面元的名称
group_names = ['Youth', 'YoungAdult', 'MiddleAged', 'Senior']
pd.cut(ages, bins, labels=group_names)

[Youth, Youth, Youth, YoungAdult, Youth, ..., YoungAdult, Senior, MiddleAged, MiddleAged, YoungAdult]
Length: 12
Categories (4, object): [Youth < YoungAdult < MiddleAged < Senior]

- 如果向cut传入的是面元的数量而不是确切的面元边界，则它会根据数据的最小值和最大值计算等长面元

In [72]:
data = np.random.rand(20)
data

array([0.96390509, 0.86930006, 0.59810395, 0.65047637, 0.58615639,
       0.80737248, 0.9917837 , 0.66293006, 0.75323444, 0.37262289,
       0.90822699, 0.44370971, 0.58329079, 0.7381706 , 0.45494561,
       0.53543332, 0.66847853, 0.65456137, 0.85511838, 0.89732591])

In [73]:
pd.cut(data, 4, precision=2)

[(0.84, 0.99], (0.84, 0.99], (0.53, 0.68], (0.53, 0.68], (0.53, 0.68], ..., (0.53, 0.68], (0.53, 0.68], (0.53, 0.68], (0.84, 0.99], (0.84, 0.99]]
Length: 20
Categories (4, interval[float64]): [(0.37, 0.53] < (0.53, 0.68] < (0.68, 0.84] < (0.84, 0.99]]

- qcut是一个非常类似于cut的函数，它可以根据样本分位数对数据进行面元划分，根据数据的分布情况，cut可能无法使各个面元中含有相同数量的数据点，而qcut使用的是样本分位数，因此可以得到大小基本相等的面元

In [74]:
data = np.random.randn(1000)
cats = pd.qcut(data, 4)
cats

[(0.617, 3.24], (0.617, 3.24], (0.617, 3.24], (0.617, 3.24], (-0.0254, 0.617], ..., (-0.0254, 0.617], (0.617, 3.24], (-0.0254, 0.617], (-3.213, -0.717], (-0.717, -0.0254]]
Length: 1000
Categories (4, interval[float64]): [(-3.213, -0.717] < (-0.717, -0.0254] < (-0.0254, 0.617] < (0.617, 3.24]]

In [75]:
cats.value_counts()

(-3.213, -0.717]     250
(-0.717, -0.0254]    250
(-0.0254, 0.617]     250
(0.617, 3.24]        250
dtype: int64

### 检测和过滤异常值

- 异常值（outlier）的过滤或变换运算在很大程度上其实就是数组运算

In [76]:
np.random.seed(12345)

In [77]:
data = DataFrame(np.random.randn(1000, 4))
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067684,0.067924,0.025598,-0.002298
std,0.998035,0.992106,1.006835,0.996794
min,-3.428254,-3.548824,-3.184377,-3.745356
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.366626,2.653656,3.260383,3.927528


In [78]:
# 选出含有“绝对值大于3的值”的行
data[(np.abs(data) > 3).any(1)]

,0,1,2,3
5,-0.539741,0.476985,3.248944,-1.021228
97,-0.774363,0.552936,0.106061,3.927528
102,-0.655054,-0.565230,3.176873,0.959533
305,-2.315555,0.457246,-0.025907,-3.399312
324,0.050188,1.951312,3.260383,0.963301
400,0.146326,0.508391,-0.196713,-3.745356
499,-0.293333,-0.242459,-3.056990,1.918403
523,-3.428254,-0.296336,-0.439938,-0.867165
586,0.275144,1.179227,-3.184377,1.369891
808,-0.362528,-3.548824,1.553205,-2.186301


In [79]:
# 将所有值限定在-3和3之间
data[np.abs(data) > 3] = np.sign(data) * 3
data.describe()

,0,1,2,3
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,-0.067623,0.068473,0.025153,-0.002081
std,0.995485,0.990253,1.003977,0.989736
min,-3.000000,-3.000000,-3.000000,-3.000000
25%,-0.774890,-0.591841,-0.641675,-0.644144
50%,-0.116401,0.101143,0.002073,-0.013611
75%,0.616366,0.780282,0.680391,0.654328
max,3.000000,2.653656,3.000000,3.000000


### 排列和随即采样

- 利用numpy.random.permutation函数可以轻松实现对Series或DataFrame的列的排列工作（permuting，随即重排序），通过需要排列的轴的长度调用permutation，可产生一个表示新顺序的整数数组

In [80]:
df = DataFrame(np.arange(5 * 4).reshape(5, 4))
df

,0,1,2,3
0,0,1,2,3
1,4,5,6,7
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


In [81]:
sampler = np.random.permutation(5)
sampler

array([1, 0, 2, 3, 4])

In [82]:
df.take(sampler)

,0,1,2,3
1,4,5,6,7
0,0,1,2,3
2,8,9,10,11
3,12,13,14,15
4,16,17,18,19


### 计算指标/哑变量

- 另一种常用于统计建模或机器学习的转换方式是：将分类变量（categorical variable）转换为“哑变量矩阵”（dummy matrix）或“指标矩阵”（indicator matrix），如果DataFrame的某一列中含有k个不同的值，则可以派生出一个k列矩阵或DataFrame（其值全为1和0）

In [83]:
df = DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df

,data1,key
0,0,b
1,1,b
2,2,a
3,3,c
4,4,a
5,5,b


In [84]:
pd.get_dummies(df['key'])

,a,b,c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


In [85]:
# 可以加上一个前缀
dummies = pd.get_dummies(df['key'], prefix='key')
dummies

,key_a,key_b,key_c
0,0,1,0
1,0,1,0
2,1,0,0
3,0,0,1
4,1,0,0
5,0,1,0


## 字符串操作

- pandas能够对整租数据应用字符串表达式和正则表达式，而且可以处理缺失数据

In [86]:
data = {'Dave': 'dave@google.com', 'Steve': 'steve@gmail.com', 'Rob': 'rob@gmail.com', 'Wes': np.nan}
data = Series(data)
data

Dave     dave@google.com
Rob        rob@gmail.com
Steve    steve@gmail.com
Wes                  NaN
dtype: object

- 通过data.map，所有字符串和正则表达式方法都能被应用于各个值，但是存在NA就会报错，为了解决这个问题，通过Series的str属性即可访问这些方法

In [87]:
data.str.contains('gmail')

Dave     False
Rob       True
Steve     True
Wes        NaN
dtype: object

In [88]:
# 也可以使用正则表达式
import re
pattern = '([A-Z0-9._%+-]+)@([A-Z0-9.-]+)\\.([A-Z]{2,4})'
matches = data.str.findall(pattern, flags=re.IGNORECASE)
matches

Dave     [(dave, google, com)]
Rob        [(rob, gmail, com)]
Steve    [(steve, gmail, com)]
Wes                        NaN
dtype: object

- 有两个办法实现矢量化的元素获取操作：要么使用str.get，要么在str属性上使用索引

In [89]:
data.str.get(0)

Dave       d
Rob        r
Steve      s
Wes      NaN
dtype: object

In [90]:
data.str[1]

Dave       a
Rob        o
Steve      t
Wes      NaN
dtype: object

In [91]:
# 对字符串进行子串截取
data.str[:5]

Dave     dave@
Rob      rob@g
Steve    steve
Wes        NaN
dtype: object